#### Import Libraries

In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, TrainerCallback

#### Load Data

In [2]:
# file paths for training and validation data
base_path = "./all_data"
train_input_file = base_path + "/train/radiology/semtypes.txt"
train_target_file = base_path + "/train/radiology/captions.txt"
valid_input_file = base_path + "/validation/radiology/semtypes.txt"
valid_target_file = base_path + "/validation/radiology/captions.txt"

train_input_data = [line.split() for line in open(train_input_file)]
train_input_data = [[t[0], " ".join(t[1:])] for t in train_input_data]
train_input_data = pd.DataFrame(train_input_data)
train_input_data.columns = ["id", "semtypes"]

train_target_data = pd.read_csv(train_target_file, sep="\t", header=None)
train_target_data.columns = ["id", "caption"]

valid_input_data = [line.split() for line in open(valid_input_file)]
valid_input_data = [[t[0], " ".join(t[1:])] for t in valid_input_data]
valid_input_data = pd.DataFrame(valid_input_data)
valid_input_data.columns = ["id", "semtypes"]

valid_target_data = pd.read_csv(valid_target_file, sep="\t", header=None)
valid_target_data.columns = ["id", "caption"]

In [3]:
import json

f = open(base_path + '/llm_result.txt', "r")
contents = f.read()
contents = contents.replace("\n", "")
json_data = json.loads(contents)

llm_df = pd.DataFrame(json_data)

llm_df = llm_df.drop('index', axis=1)

llm_df

,id,relationship,summary
0,ROCO_00002,with a mass of homogeneous attenuation (necro...,The patient has undergone a CT scan which sho...
1,ROCO_00003,The patient developed pain and discomfort on ...,The patient developed pain and discomfort on ...
2,ROCO_00004,,The UMLS Metathesaurus has 12435 concepts in ...
3,ROCO_00005,,The UMLS Metathesaurus has 12456 semantic typ...
4,ROCO_00007,The given UMLS semantic types are 1. Radiogra...,There is no single UMLS semantic type that re...
...,...,...,...
1066,ROCO_01329,The answer is Causative Agent: Bacterium.,\nCausative Agent: Bacterium
1067,ROCO_01330,and ascites with dilated veins at the periphe...,No new information.
1068,ROCO_01331,,The UMLS Metathesaurus has 12435 concepts in ...
1069,ROCO_01332,".\nThe UMLS semantic types are 1. Body Part, O...",The given UMLS semantic types are 1. Diagnost...


#### Filter ones that have been training by the LLM

In [4]:
train_target_data = train_target_data.merge(llm_df, on='id')
train_target_data

,id,caption,relationship,summary
0,ROCO_00002,Computed tomography scan in axial view showin...,with a mass of homogeneous attenuation (necro...,The patient has undergone a CT scan which sho...
1,ROCO_00003,Bacterial contamination occurred after comple...,The patient developed pain and discomfort on ...,The patient developed pain and discomfort on ...
2,ROCO_00004,The patient had residual paralysis of the han...,,The UMLS Metathesaurus has 12435 concepts in ...
3,ROCO_00005,Panoramic radiograph after immediate loading.,,The UMLS Metathesaurus has 12456 semantic typ...
4,ROCO_00007,Plain abdomen x-ray: Multiple air levels at t...,The given UMLS semantic types are 1. Radiogra...,There is no single UMLS semantic type that re...
...,...,...,...,...
1066,ROCO_01329,Abdominal CT finding. Enterocutaneous fistula...,The answer is Causative Agent: Bacterium.,\nCausative Agent: Bacterium
1067,ROCO_01330,(Case 2) Post operative CT scan showing persi...,and ascites with dilated veins at the periphe...,No new information.
1068,ROCO_01331,Magnetic resonance image showing the patient'...,,The UMLS Metathesaurus has 12435 concepts in ...
1069,ROCO_01332,Post-operative chest X-ray image of the same ...,".\nThe UMLS semantic types are 1. Body Part, O...",The given UMLS semantic types are 1. Diagnost...


In [5]:
train_input_data = train_input_data[train_input_data['id'].isin(train_target_data['id'])]
train_input_data

,id,semtypes
0,ROCO_00002,T060 Diagnostic Procedure T061 Therapeutic or ...
1,ROCO_00003,T061 Therapeutic or Preventive Procedure T058 ...
2,ROCO_00004,T060 Diagnostic Procedure T047 Disease or Synd...
3,ROCO_00005,T060 Diagnostic Procedure T170 Intellectual Pr...
4,ROCO_00007,T109 Organic Chemical T074 Medical Device T121...
...,...,...
1066,ROCO_01329,T029 Body Location or Region
1067,ROCO_01330,T033 Finding T060 Diagnostic Procedure T046 Pa...
1068,ROCO_01331,T033 Finding T047 Disease or Syndrome T170 Int...
1069,ROCO_01332,T047 Disease or Syndrome T040 Organism Functio...


In [6]:
from sklearn.model_selection import train_test_split

# Split train_input_data into train, test, and valid datasets
train_input, valid_test_input = train_test_split(train_input_data, test_size=0.4, random_state=42)
valid_input, test_input = train_test_split(valid_test_input, test_size=0.5, random_state=42)

# Split train_target_data into train, test, and valid datasets
train_target, valid_test_target = train_test_split(train_target_data, test_size=0.4, random_state=42)
valid_target, test_target = train_test_split(valid_test_target, test_size=0.5, random_state=42)

In [7]:
# Reset index
train_input = train_input.reset_index(drop=True)
valid_input = valid_input.reset_index(drop=True)
test_input = test_input.reset_index(drop=True)

train_target = train_target.reset_index(drop=True)
valid_target = valid_target.reset_index(drop=True)
test_target = test_target.reset_index(drop=True)

In [8]:
# Select n% of data
n = 20
n = n / 100
train_input = train_input.sample(frac=n, random_state=42)
valid_input = valid_input.sample(frac=n, random_state=42)
test_input = test_input.sample(frac=n, random_state=42)

train_target = train_target.sample(frac=n, random_state=42)
valid_target = valid_target.sample(frac=n, random_state=42)
test_target = test_target.sample(frac=n, random_state=42)

In [9]:
valid_target

,id,caption,relationship,summary
9,ROCO_00272,Chest radiography from case 1 showing a pneum...,,The UMLS Metathesaurus has 12435 concepts in ...
197,ROCO_01135,Transabdominal sonographic image of liver of ...,\nThe UMLS semantic types corresponding to thi...,The given diagnosis is Transabdominal sonogr...
66,ROCO_00125,Abdominal computed tomography reveals an ill-...,,The UMLS Metathesaurus has 12435 concepts in ...
191,ROCO_00283,Magnetic resonance scan detailing radially lo...,The relationships between the given UMLS sema...,A medical assistant reviewing the provided in...
117,ROCO_00295,Metapex intracanal medicament,.,The UMLS Metathesaurus has 27413 concepts in ...
111,ROCO_01330,(Case 2) Post operative CT scan showing persi...,and ascites with dilated veins at the periphe...,No new information.
15,ROCO_00539,MRI showing the hypointense lobulated mass in...,This is because a medical assistant who gives...,"The UMLS semantic type ""Diagnosis"" can be ass..."
86,ROCO_00896,Preoperative findings.Coronal view of contras...,"Fluid density was predominantly hypoechoic, s...",The given UMLS semantic types are 1. Finding ...
75,ROCO_01203,Saphenous nerve is located outside of pes ans...,The UMLS semantic types are in the form of...,
144,ROCO_00688,An 8-week fetus associated with a large (at l...,The gestational age at time of image acquisit...,The gestational age at time of image acquisit...


#### T5 tokenizer and model

In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

#### Custom dataset class

In [11]:
class RoCoDataset(Dataset):
    def __init__(self, input_file, target_file, tokenizer):
        self.data = pd.merge(input_file, target_file, on="id", how="inner")
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        input_text = row['semtypes']
        target_text = f"{row['caption']} </s> {row['relationship']} </s> {row['summary']}"
        
#         input_encoding = self.tokenizer.encode_plus(input_text, padding=True, truncation=True)
#         target_encoding = self.tokenizer.encode_plus(target_text, padding=True, truncation=True)

        input_encoding = self.tokenizer.encode_plus(input_text, padding='max_length', max_length=1024, truncation=True)
        target_encoding = self.tokenizer.encode_plus(target_text, padding='max_length', max_length=1024, truncation=True)

        input_ids = input_encoding['input_ids']
        input_attention_mask = input_encoding['attention_mask']
        target_ids = target_encoding['input_ids']
        target_attention_mask = target_encoding['attention_mask']
        
        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(input_attention_mask),
            'labels': torch.tensor(target_ids),
            'decoder_attention_mask': torch.tensor(target_attention_mask),
        }

#### Load dataset in custom format

In [12]:
train_dataset = RoCoDataset(train_input, train_target, tokenizer)
valid_dataset = RoCoDataset(valid_input, valid_target, tokenizer)

#### Define the training arguments and create the trainer

In [15]:
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=20,
    save_total_limit=2,
    overwrite_output_dir=True,
    learning_rate=1e-4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=20,
    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

In [ ]:
# Train the model
trainer.train()

# Save the trained model
trainer.save_model('./trained_model')

Step,Training Loss,Validation Loss


In [ ]:
for _, row in test_input.iterrows():
    input_id = row['id']
    input_text = row['semtypes']
    
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate captions using the model
    outputs = model.generate(input_ids)

    # Decode the generated captions
    generated_captions = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    target_caption = test_target[test_target['id'] == input_id]['caption'].values[0]
    print('Input: ', input_text)
    print('Generated Caption: ', generated_captions)
    print('Original Caption: ', target_caption)
    print()

In [ ]:
# # file paths for training and validation data
# base_path = "./roco-image-captioning/all_data"
# train_input_file = base_path + "/train/radiology/semtypes.txt"
# train_target_file = base_path + "/train/radiology/captions.txt"
# valid_input_file = base_path + "/validation/radiology/semtypes.txt"
# valid_target_file = base_path + "/validation/radiology/captions.txt"

# train_input_data = [line.split() for line in open(train_input_file)]
# train_input_data = [[t[0], " ".join(t[1:])] for t in train_input_data]
# train_input_data = pd.DataFrame(train_input_data)
# train_input_data.columns = ["id", "semtypes"]

# train_target_data = pd.read_csv(train_target_file, sep="\t", header=None)
# train_target_data.columns = ["id", "caption"]

# # last 10 rows of train_input_data
# input_data = train_input_data.tail(50)
# target_data = train_target_data.tail(50)

# for _, row in input_data.iterrows():
#     input_id = row['id']
#     input_text = row['semtypes']
    
#     # Tokenize the input text
#     input_ids = tokenizer.encode(input_text, return_tensors='pt')

#     # Generate captions using the model
#     outputs = model.generate(input_ids)

#     # Decode the generated captions
#     generated_captions = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#     target_caption = train_target_data[train_target_data['id'] == input_id]['caption'].values[0]
#     print('Input: ', input_text)
#     print('Generated Caption: ', generated_captions)
#     print('Original Caption: ', target_caption)
#     print()